<a href="https://colab.research.google.com/github/zhdbw1022/pytorch_alexnet/blob/master/alexnet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorboardX
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tensorboardX import SummaryWriter
import numpy as np
import os
import _pickle as cPickle
from PIL import Image
import matplotlib.pyplot as plt








     # 定义全局变量
modelPath = './model.pkl'
batchSize = 5
nEpochs = 20
numPrint = 1000
# 定义Summary_Writer
writer = SummaryWriter('./Result')   # 数据存放在这个文件夹
# cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.Resize(size=(227, 227)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
    normalize
])

# 加载数据集 (训练集和测试集)
trainset = torchvision.datasets.CIFAR10(root='./Cifar-10', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./Cifar-10', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchSize, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batchSize, shuffle=False)



#CURRENT_DIR = os.getcwd()
# 
#
#def read_cifar10_train_data(dataset_file_path):
#    data_dir = dataset_file_path
#    train_name = 'data_batch_'
#    train_X = None
#    train_Y = None
#
#    # train data
#    for i in range(1,6):
#        file_path = data_dir+train_name+str(i)
#        with open(file_path, 'rb') as fo:
#            
#            dict = cPickle.load(fo)
#           
#            if  train_X is None:
#                train_X = dict['data']
#                train_Y = dict['labels']
#            else:
#                train_X = np.concatenate((train_X, dict['data']), axis=0)
#                train_Y = np.concatenate((train_Y, dict['labels']), axis=0)
#                
# 
#    
#    train_X = train_X.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
#
#    train_y_vec = np.zeros((len(train_Y), 10), dtype=np.float)
#
#    for i, label in enumerate(train_Y):
#        train_y_vec[i, int(train_Y[i])] = 1.  # y_vec[1,3] means #2 row, #4column
#
# 
#    return train_X, train_y_vec
#
#
#def read_cifar10_test_data(dataset_file_path):
#    data_dir = dataset_file_path
#    test_name = 'test_batch'
#    test_X = None
#    test_Y = None
#      
#    # test_data
#    file_path = data_dir + test_name
#    with open(file_path, 'rb') as fo:
#        dict =cPickle.load(fo,encoding='iso-8859-1')
# 
#        test_X = dict['data']
#        test_Y = dict['labels']
#
#    test_X = test_X.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
#
#    test_y_vec = np.zeros((len(test_Y), 10), dtype=np.float)
#
#    for i, label in enumerate(test_Y):
#        test_y_vec[i, int(test_Y[i])] = 1.  # y_vec[1,3] means #2 row, #4column
# 
#    return test_X, test_y_vec



#class BatchReadData(object):
#
#    def __init__(self, dataset_file_path, output_size=[227, 227], train_data=True, shuffle=False):
#        self.output_size = output_size
#        self.shuffle = shuffle
#
#        self.pointer = 0
#        # 读数据
#        if train_data:
#            self.images, self.labels = read_cifar10_train_data(dataset_file_path)
#        else:
#            self.images, self.labels = read_cifar10_test_data(dataset_file_path)
#        
#        # Shuffle the data
#        if self.shuffle:
#            self.shuffle_data()
#            
#            
#    def reset_pointer(self):
#        self.pointer = 0
#        
#        if self.shuffle:
#            self.shuffle_data()
#            
#    def shuffle_data(self):
#        temp_images = self.images[:]
#        temp_labels = self.labels[:]
#        
#        self.images = []
#        self.labels = []
#
#        idx = np.random.permutation(len(temp_labels))
#        for i in idx:
#            self.images.append(temp_images[i])
#            self.labels.append(temp_labels[i])
#            
#            
#    def next_batch(self, batch_size):
#        # Get next batch of image (path) and labels
#        paths = self.images[self.pointer:(self.pointer+batch_size)]
#        labels = self.labels[self.pointer:(self.pointer+batch_size)]        
#        
#        print (len(paths))
#        print (paths[0].shape)
#        
#        # Update pointer
#        self.pointer += batch_size
#        
#        # Read images
#        images = np.ndarray([batch_size, self.output_size[0], self.output_size[1], 3])
#        #images = np.zeros((batch_size, self.output_size[0], self.output_size[1], 3))
#        
#        for i in range(len(paths)):
#            
#            img = paths[i]           
#
#            # Resize the image for output
#            img = Image.fromarray(img)
#            img = np.array(img.resize((227,227),Image.BICUBIC))# 修改分辨率，再转为array类
#            #img = cv2.resize(img, (self.output_size[0], self.output_size[0]))     #  这上面的两种方法都可以
#            images[i,:,:,:] = img
#
#        return images/255., labels
#    # 测试代码 
#dataset_file_path = CURRENT_DIR+'/Cifar-10/cifar-10-batches-py/'
#
#
#one = BatchReadData(dataset_file_path, [227, 227],False, False)
#
#for i in range(10):
#     
#    images, labels = one.next_batch(100)
#     
#    fig, axarr = plt.subplots(1, 2) 
#    axarr[0].imshow(images[0]) 
#    axarr[1].imshow(images[1])
#    print (labels[0], labels[1])
#    plt.show()
#    if i==3:
#        one.reset_pointer()



class AlexNet(nn.Module):
    def __init__(self,num_classes=10):
        super(AlexNet,self).__init__()
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4,padding=2,bias=False),
            nn.BatchNorm2d(96),
            nn.ReLU(True),
 
            nn.MaxPool2d(kernel_size=3,stride=2,padding=0),
            nn.Conv2d(in_channels=96,out_channels=192,kernel_size=5,stride=1,padding=2,bias=False),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
 
            nn.MaxPool2d(kernel_size=3,stride=2,padding=0),
            nn.Conv2d(in_channels=192,out_channels=384,kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(384),
            nn.ReLU(True),
 
            nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
 
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
 
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=256*6*6,out_features=4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(True),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
    def forward(self,x):
        x = self.feature_extraction(x)
        x = x.view(x.size(0),256*6*6)
        x = self.classifier(x)
        return x


net = AlexNet().to(device)

# 使用测试数据测试网络
def Accuracy():
    correct = 0
    total = 0
    with torch.no_grad():  # 训练集中不需要反向传播
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device) # 将输入和目标在每一步都送入GPU
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)  # 返回每一行中最大值的那个元素，且返回其索引
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    return 100.0 * correct / total

# 训练函数
def train():
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  # 随机梯度下降
    iter = 0
    num = 1
    # 训练网络
    for epoch in range(nEpochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            iter = iter + 1
            # 取数据
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)  # 将输入和目标在每一步都送入GPU
            # 将梯度置零
            optimizer.zero_grad()
            # 训练
            outputs = net(inputs)
            loss = criterion(outputs, labels).to(device)
            loss.backward()   # 反向传播
            writer.add_scalar('loss', loss.item(), iter)
            optimizer.step()  # 优化
            # 统计数据
            running_loss += loss.item()
            if i % numPrint == 999:    # 每 batchsize * numPrint 张图片，打印一次
                print('epoch: %d\t batch: %d\t loss: %.6f' % (epoch + 1, i + 1, running_loss / (batchSize*numPrint)))
                running_loss = 0.0
                writer.add_scalar('accuracy', Accuracy(), num + 1)
                num = num + 1
    # 保存模型
    torch.save(net, './model.pkl')

if __name__ == '__main__':
    # 如果模型存在，加载模型
    if os.path.exists(modelPath):
        print('model exits')
        net = torch.load(modelPath)
        print('model loaded')
    else:
        print('model not exits')
    print('Training Started')
    train()
    writer.close()
    print('Training Finished')

     |████████████████████████████████| 194kB 3.4MB/s 


0it [00:00, ?it/s]

170500096it [00:02, 57868858.61it/s]                               


Extracting ./Cifar-10/cifar-10-python.tar.gz to ./Cifar-10
Files already downloaded and verified
model not exits
Training Started
epoch: 1	 batch: 1000	 loss: 0.409240
Accuracy of the network on the 10000 test images: 30 %
epoch: 1	 batch: 2000	 loss: 0.362020
Accuracy of the network on the 10000 test images: 38 %
epoch: 1	 batch: 3000	 loss: 0.334070
Accuracy of the network on the 10000 test images: 39 %
epoch: 1	 batch: 4000	 loss: 0.328885
Accuracy of the network on the 10000 test images: 42 %
epoch: 1	 batch: 5000	 loss: 0.313853
Accuracy of the network on the 10000 test images: 43 %
epoch: 1	 batch: 6000	 loss: 0.303270
Accuracy of the network on the 10000 test images: 43 %
epoch: 1	 batch: 7000	 loss: 0.301450
Accuracy of the network on the 10000 test images: 43 %
epoch: 1	 batch: 8000	 loss: 0.292954
Accuracy of the network on the 10000 test images: 46 %
epoch: 1	 batch: 9000	 loss: 0.281095
Accuracy of the network on the 10000 test images: 49 %
epoch: 1	 batch: 10000	 loss: 0.2